In [10]:
%matplotlib widget
import ipywidgets
from ipywidgets import widgets

from ipywidgets.widgets import VBox,HBox
from IPython.display import display
# import numpy as np
import pandas as pd

from matplotlib import pyplot as plt

import nmrExpt

# import nmrglue as ng
import tempfile
import io

In [11]:
# %matplotlib inline

### Global lists and dictionaries

In [12]:
nmrExpts = {}
upload_file_names = ["proton1D", "carbon1D", "cosy2D", "hsqc2D", "hmbc2D", "peakTable"]
upload_file_widgets = {}
upload_button_status = [""]




### Define All Widgets

In [13]:
upload_1D_proton  = ipywidgets.widgets.FileUpload(multiple=True, 
                                                  description="1D Proton",
                                                  description_tooltip="Upload a 1D proton NMR fid\nif varian, choose procar and fid simutaneously")

upload_1D_carbon  = ipywidgets.widgets.FileUpload(multiple=True, 
                                                  description="1D Carbon",
                                                  description_tooltip="Upload a 1D carbon NMR fid\nif varian, choose procar and fid simutaneously")

upload_2D_cosy    = ipywidgets.widgets.FileUpload(multiple=True, 
                                                  description="2D Cosy",
                                                  description_tooltip="Upload a 2D COSY NMR fid\nif varian, choose procar and fid simutaneously")

upload_2D_hsqc    = ipywidgets.widgets.FileUpload(multiple=True, 
                                                  description="2D HSQC",
                                                  description_tooltip="Upload a 2D HSQC NMR fid\nif varian, choose procar and fid simutaneously")

upload_2D_hmbc    = ipywidgets.widgets.FileUpload(multiple=True, 
                                                  description="2D HMBC",
                                                  description_tooltip="Upload a 2D HMBC NMR spectrum\n if varian, choose procar and fid simutaneously")

upload_peak_table = ipywidgets.widgets.FileUpload(multiple=True, 
                                                  description="Peak Tables",
                                                  description_tooltip="Upload MNOVA generated excel peak table ")


varianBrukerRadioButtons = widgets.RadioButtons( options=['varian', 'bruker'],
                                                 value='varian', 
                                                 layout={'width': '150px'}, # If the items' names are long
                                                 description='Input Source',
                                                 disabled=False
                                                            )

output2 = ipywidgets.widgets.Output()


upload_file_widgets["proton1D"] = { "nmrExptKey"   : "proton1D",
                                    "uploadButton" : upload_1D_proton }

upload_file_widgets["carbon1D"] = { "nmrExptKey"   : "carbon1D",
                                   "uploadButton" : upload_1D_carbon }

upload_file_widgets["cosy2D"] = { "nmrExptKey"   : "cosy2D",
                                  "uploadButton" : upload_2D_cosy }

upload_file_widgets["hsqc2D"] = { "nmrExptKey"   : "hsqc2D",
                                  "uploadButton" : upload_2D_hsqc }

upload_file_widgets["hmbc2D"] = { "nmrExptKey"   : "hmbc2D",
                                  "uploadButton" : upload_2D_hmbc }

upload_file_widgets["peakTable"] = { "nmrExptKey"   : "peakTable",
                                     "uploadButton" : upload_peak_table }



upload_buttons = [ widgets.Button(description=b) for b in upload_file_names ]
btn_output = widgets.Output()


mpl_output = widgets.Output(height="600px",width="600px")

phase_output = widgets.Output()

slider_width = '350px'
phse_ftext_width = '75px'

rp0_slider = widgets.FloatSlider(description='rp0', 
                                 value=0,
                                 min=-360.0,
                                 max=360.0,
                                 step=0.1,
                                 continuous_update=False,
                                 layout={'width': slider_width})

rp0_ftext  = widgets.FloatText(  value=0, step=0.1, layout={'width': phse_ftext_width})

lp0_slider = widgets.FloatSlider(description='lp0', 
                                 value=0,
                                 min=-360.0,
                                 max=360.0,
                                 step=0.1,
                                 continuous_update=False,
                                 layout={'width': slider_width})

lp0_ftext  = widgets.FloatText(  value=0, step=0.1, layout={'width': phse_ftext_width})


rp1_slider = widgets.FloatSlider(description='rp1', 
                                 value=0,
                                 min=-360.0,
                                 max=360.0,
                                 step=0.1,
                                 continuous_update=False,
                                 layout={'width': slider_width})

rp1_ftext  = widgets.FloatText(  value=0, step=0.1, layout={'width': phse_ftext_width})

lp1_slider = widgets.FloatSlider(description='lp1', 
                                 value=0,
                                 min=-360.0,
                                 max=360.0,
                                 step=0.1,
                                 continuous_update=False,
                                 layout={'width': slider_width})

lp1_ftext  = widgets.FloatText( value=0, step=0.1, layout={'width': phse_ftext_width})


### Create Buttons to Display Spectra to phase correct

In [14]:


    
# upload_buttons = [ widgets.Button(description=b) for b in upload_file_names ]

def on_button_clicked(clicked):
    
    #set all buttons background to light gray
    for b in upload_buttons:
        b.style.button_color = 'lightgray'
    
    i = upload_file_names.index(clicked.description)
    
    upload_button_status[0]=clicked.description
    
    upload_buttons[i].style.button_color = 'lightgreen'
    
    if clicked.description in ['proton1D', 'carbon1D']:
    
        nmrExpts[clicked.description].plot_1D(fig=nmrfig,ax=ax1D1H)
    
        # set phase slider values from current spectra for 1D data
        
        rp0_slider.value =  nmrExpts[clicked.description].udic[0]['rp']
        lp0_slider.value =  nmrExpts[clicked.description].udic[0]['lp']
        
        rp0_ftext.value = nmrExpts[clicked.description].udic[0]['rp']
        lp0_ftext.value = nmrExpts[clicked.description].udic[0]['lp']
        
    elif clicked.description in ['cosy2D', 'hsqc2D', 'hmbc2D']:
        
        nmrExpts[clicked.description].plot_2D(fig=nmrfig,ax=ax1D1H)
    
        # set phase slider values from current spectra for 2D data
        
        rp0_slider.value =  nmrExpts[clicked.description].udic[1]['rp']
        lp0_slider.value =  nmrExpts[clicked.description].udic[1]['lp']
        
        rp0_ftext.value = nmrExpts[clicked.description].udic[1]['rp']
        lp0_ftext.value = nmrExpts[clicked.description].udic[1]['lp']
        
        rp1_slider.value =  nmrExpts[clicked.description].udic[0]['rp']
        lp1_slider.value =  nmrExpts[clicked.description].udic[0]['lp']
        
        rp1_ftext.value = nmrExpts[clicked.description].udic[0]['rp']
        lp1_ftext.value = nmrExpts[clicked.description].udic[0]['lp']
        
    
#     with btn_output:
#         print("description", type(clicked),clicked.description, upload_file_names.index(clicked.description))
        
for b in upload_buttons:
    b.on_click(   lambda clicked: on_button_clicked(clicked))
    b.style.button_color = 'lightgray'

### create buttons to upload NMR data

In [15]:

def on_value_change(change, nmrExpts, nmrExptKey):
    
#     print("nmrExptKey", nmrExptKey)
    
#     upload_file_widgets[nmrExptKey]["completedTick"].value = False
    upload_file_widgets[nmrExptKey]["uploadButton"].style.button_color = 'lightgray'
    
    if nmrExptKey == "peakTable":
        try:
            for k in change['new']:
                nmrExpts[nmrExptKey] = pd.read_excel(io.BytesIO(change['new'][k]["content"]), sheet_name=None, index_col=0)

#             with output2:
#                     print(change['new'][k].keys())
                    
            

#             if nmrExptKey == "peakTable":
#                 upload_file_widgets[nmrExptKey]["completedTick"].value = True
                
            upload_file_widgets[nmrExptKey]["uploadButton"].style.button_color = 'lightgreen'
            on_button_clicked(upload_buttons[upload_file_names.index(nmrExptKey)])
        except:
#             upload_file_widgets[nmrExptKey]["completedTick"].value = False
            upload_file_widgets[nmrExptKey]["uploadButton"].style.button_color = 'lightgray'
            
    
    else:
        
        #if 'procpar' in change['new'] and 'fid' in change['new']:
            try:
#                 print("read in nmr file")
                procpar_file = change['new']['procpar']
                fid_file     = change['new']['fid']

                f, fprocpar =  tempfile.mkstemp( text=True) 

                with open(fprocpar, "wb" ) as f:
                    f.write(procpar_file["content"])

                f, ffid =  tempfile.mkstemp( text=True) 

                with open(ffid, "wb" ) as f:
                    f.write(fid_file["content"])

                nmrExpts[nmrExptKey] = nmrExpt.NMRexpt.from_varian(fid_file=ffid, procpar_file=fprocpar)
                
                if nmrExpts[nmrExptKey].udic['ndim'] == 1:
                
                    nmrExpts[nmrExptKey].produce_1D(lb = [0.001])
                
                else:
                    nmrExpts[nmrExptKey].produce_2D()


#                 upload_file_widgets[nmrExptKey]["completedTick"].value = True
                upload_file_widgets[nmrExptKey]["uploadButton"].style.button_color = 'lightgreen'
                on_button_clicked(upload_buttons[upload_file_names.index(nmrExptKey)])
        
#                 with output2:
#                     print("read in nmr file", change['owner'].description, nmrExpts[nmrExptKey].data.shape, change['new'].keys())

            except:
#                 upload_file_widgets[nmrExptKey]["completedTick"].value = False
                upload_file_widgets[nmrExptKey]["uploadButton"].style.button_color = 'lightgray'
#                 with output2:
#                     print("Exception Occured: Wrong File Type")
                
            

    
        
upload_1D_proton.observe(   lambda change: on_value_change(change, nmrExpts, 'proton1D'),  names='value')
upload_1D_carbon.observe(   lambda change: on_value_change(change, nmrExpts, 'carbon1D'),  names='value')
upload_2D_cosy.observe(     lambda change: on_value_change(change, nmrExpts, 'cosy2D'),    names='value')
upload_2D_hsqc.observe(     lambda change: on_value_change(change, nmrExpts, 'hsqcq2D'),   names='value')
upload_2D_cosy.observe(     lambda change: on_value_change(change, nmrExpts, 'hmbc2D'),    names='value')
upload_2D_cosy.observe(     lambda change: on_value_change(change, nmrExpts, 'hmbc2D'),    names='value')
upload_peak_table.observe(  lambda change: on_value_change(change, nmrExpts, 'peakTable'), names='value')

### Create NMR plot window

In [16]:

with mpl_output:

    nmrfig = plt.figure(figsize=(8,6))
    ax1D1H = plt.axes([0.1,0.1,0.8,0.8]) #
    
lineplot, = ax1D1H.plot([1,2,3],[1,2,3])

### Phase sliders and values

In [17]:
# Phase sliders and direct input



# phase_vbox = widgets.VBox([ widgets.HBox([rp0_slider,rp0_ftext]),
#                             widgets.HBox([lp0_slider,lp0_ftext]),
#                             widgets.HBox([rp1_slider,rp1_ftext]),
#                             widgets.HBox([lp1_slider,lp1_ftext])])

phase_hbox = HBox([
                    VBox([
                           HBox([rp0_slider,rp0_ftext]),
                           HBox([lp0_slider,lp0_ftext])
                    ]),
                    VBox([
                           HBox([rp1_slider,rp1_ftext]),
                           HBox([lp1_slider,lp1_ftext])
                    ])
])

### Link phase sliders to float input boxes

link_rp0 = widgets.jslink((rp0_slider, 'value'), (rp0_ftext, 'value'))
link_lp0 = widgets.jslink((lp0_slider, 'value'), (lp0_ftext, 'value'))

link_rp1 = widgets.jslink((rp1_slider, 'value'), (rp1_ftext, 'value'))
link_lp1 = widgets.jslink((lp1_slider, 'value'), (lp1_ftext, 'value'))


### Create a callback to update screen

def on_phase_changed(change, widgetname):
    
    # collect values
    
    rp0 = rp0_slider.value
    lp0 = lp0_slider.value
    rp1 = rp1_slider.value
    lp1 = lp1_slider.value
    
    if upload_button_status[0] in ['proton1D', 'carbon1D']:
        nmrExpts[upload_button_status[0]].produce_1D( rp = [rp0,], lp=[lp0])
        
    elif upload_button_status[0] in ['hsqc2D', 'hmbc2D']:
        nmrExpts[upload_button_status[0]].produce_2D( rp = [rp0, rp1], lp=[lp0, lp1])
    
    on_button_clicked(upload_buttons[upload_file_names.index(upload_button_status[0])])
    
#     with phase_output:
#         print("phase changed", widgetname, upload_button_status[0], type(nmrExpts[upload_button_status[0]]) )
        
### Attach callback

rp0_slider.observe(   lambda change: on_phase_changed(change, 'rp0'),  names='value')
lp0_slider.observe(   lambda change: on_phase_changed(change, 'lp0'),  names='value')

rp1_slider.observe(   lambda change: on_phase_changed(change, 'rp1'),  names='value')
lp1_slider.observe(   lambda change: on_phase_changed(change, 'lp1'),  names='value')

# HBox([rp0_slider, rp0_ftext, phase_output] )

### Create ipywidgets display

In [18]:
VBox([
        varianBrukerRadioButtons, 
        HBox([  
            upload_1D_proton,
            upload_1D_carbon,
            upload_2D_cosy,
            upload_2D_hsqc,
            upload_2D_hmbc,
            upload_peak_table
        ]),
        HBox(upload_buttons),
        phase_hbox,
        mpl_output,
#         btn_output,
#         phase_output
    
])